In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../data/01_raw/iot_sgcc_electricity.csv")

In [3]:
df.head()

,CONS_NO,FLAG,2014/1/1,2014/1/10,2014/1/11,2014/1/12,2014/1/13,2014/1/14,2014/1/15,2014/1/16,...,2016/9/28,2016/9/29,2016/9/3,2016/9/30,2016/9/4,2016/9/5,2016/9/6,2016/9/7,2016/9/8,2016/9/9
0,0387DD8A07E07FDA6271170F86AD9151,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10.12,9.96,16.92,7.60,27.22,18.05,26.47,18.75,17.84,14.92
1,01D6177B5D4FFE0CABA9EF17DAFC2B84,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,4B75AC4F2D8434CFF62DB64D0BB43103,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,B32AC8CC6D5D805AC053557AB05F5343,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.50,9.99,11.78,18.59,26.80,18.57,14.59,12.82,19.37,15.92
4,EDFC78B07BA2908B3395C4EB2304665E,1,2.9,3.42,3.81,4.58,3.56,4.25,3.86,3.53,...,17.77,10.37,15.32,13.51,12.23,14.68,16.35,18.14,18.41,17.31


In [4]:
df.shape, df["FLAG"].value_counts()

((42372, 1036),
 0    38757
 1     3615
 Name: FLAG, dtype: int64)

In [5]:
# for c in df.columns:
#     print(c)

In [6]:
data_cols = [c for c in df.columns if c not in ["CONS_NO", "FLAG"]]

In [7]:
data_cols_datetime = pd.to_datetime(data_cols)
df.columns = ["CONS_NO", "FLAG"] + data_cols_datetime.tolist()

data_cols_datetime_sorted = sorted(data_cols_datetime)
info_cols = ["CONS_NO", "FLAG"]

In [8]:
# df[sorted(data_cols_datetime)]

# Preprocessing

Implement the steps outlined in the paper

1. Fill in NaN
2. Remove outliers
3. Normalize

### Fill in NaN based on average of prev and next if both are not Null; 0 otherwose

In [9]:
data_T = df[data_cols_datetime_sorted].T

In [10]:
%%time
data_T = data_T.interpolate()
data_T.fillna(0, inplace=True)

CPU times: user 11.2 s, sys: 207 ms, total: 11.4 s
Wall time: 11.4 s


### Clip outliers based on 2 standard deviations

In [11]:
three_sigma = data_T.mean() + 2*data_T.std()

In [12]:
data_T = data_T.clip(upper=three_sigma, axis=1)

### Normalize using min-max norm

In [13]:
data_T = (data_T - data_T.min())/(data_T.max() - data_T.min() + 1e-8)

In [14]:
data_T#[[0,1,3]]#.describe()

,0,1,2,3,4,5,6,7,8,9,...,42362,42363,42364,42365,42366,42367,42368,42369,42370,42371
2014-01-01,0.000000,0.0,0.000000,0.000000,0.106331,0.000000,0.020868,0.078134,0.000000,0.103181,...,0.000000,0.487953,0.000000,0.384209,0.000000,0.000000,0.335423,0.114013,0.532178,0.000000
2014-01-02,0.000000,0.0,0.000000,0.000000,0.248451,0.000000,0.020868,0.109139,0.000000,0.074156,...,0.000000,0.525634,0.000000,0.370961,0.000000,0.000000,0.000000,0.228026,0.469056,0.000000
2014-01-03,0.000000,0.0,0.000000,0.000000,0.318474,0.000000,0.047427,0.058290,0.000000,0.078744,...,0.000000,0.516888,0.000000,0.362128,0.000000,0.000000,0.000000,0.180848,0.618879,0.000000
2014-01-04,0.000000,0.0,0.000000,0.000000,0.128116,0.000000,0.051221,0.126502,0.000000,0.090448,...,0.000000,0.344592,0.000000,0.359184,0.000000,0.000000,0.710599,0.192643,0.543061,0.000000
2014-01-05,0.000000,0.0,0.000000,0.000000,0.143158,0.000000,0.039838,0.057050,0.000000,0.092320,...,0.000000,0.388554,0.000000,1.000000,0.000000,0.000000,0.751595,0.302724,0.528913,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-10-27,0.331067,0.0,1.000000,0.684207,0.304988,0.065890,0.223853,0.439037,0.329785,0.535756,...,0.655439,0.776713,0.313463,0.499030,0.253343,0.369084,0.576430,0.100253,0.323950,0.654601
2016-10-28,0.351877,0.0,1.000000,0.590742,0.345965,0.299644,0.214368,0.306334,0.383349,0.530700,...,0.633868,0.799203,0.343556,0.576314,0.200820,0.331263,0.460896,0.155294,0.362766,0.668581
2016-10-29,0.285285,0.0,1.000000,0.633854,0.520762,0.065890,0.254206,0.332379,0.388601,0.423961,...,0.496696,0.376388,0.570503,1.000000,0.163746,0.443423,0.772715,0.129739,0.396866,0.945717
2016-10-30,0.346580,0.0,1.000000,0.475885,0.467337,0.059615,0.239030,0.420434,0.533538,0.412351,...,0.550348,0.379840,0.471449,0.705120,0.237895,0.468202,0.751595,0.076664,0.403033,0.588812


In [16]:
data_processed = data_T.T

In [17]:
data_processed["CONS_NO"] = df["CONS_NO"].values
data_processed["FLAG"] = df["FLAG"].values

In [18]:
data_processed

,2014-01-01 00:00:00,2014-01-02 00:00:00,2014-01-03 00:00:00,2014-01-04 00:00:00,2014-01-05 00:00:00,2014-01-06 00:00:00,2014-01-07 00:00:00,2014-01-08 00:00:00,2014-01-09 00:00:00,2014-01-10 00:00:00,...,2016-10-24 00:00:00,2016-10-25 00:00:00,2016-10-26 00:00:00,2016-10-27 00:00:00,2016-10-28 00:00:00,2016-10-29 00:00:00,2016-10-30 00:00:00,2016-10-31 00:00:00,CONS_NO,FLAG
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.306095,0.360579,0.207343,0.331067,0.351877,0.285285,0.346580,0.255016,0387DD8A07E07FDA6271170F86AD9151,1
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,01D6177B5D4FFE0CABA9EF17DAFC2B84,1
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.963074,4B75AC4F2D8434CFF62DB64D0BB43103,1
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.568034,0.490694,0.644715,0.684207,0.590742,0.633854,0.475885,0.385710,B32AC8CC6D5D805AC053557AB05F5343,1
4,0.106331,0.248451,0.318474,0.128116,0.143158,0.233409,0.201251,0.146789,0.139008,0.133303,...,0.395240,0.272830,0.272311,0.304988,0.345965,0.520762,0.467337,0.408207,EDFC78B07BA2908B3395C4EB2304665E,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42367,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.358651,0.392560,0.389951,0.369084,0.331263,0.443423,0.468202,0.331263,F1472871E1AFF49D4289564B6377D76C,0
42368,0.335423,0.000000,0.000000,0.710599,0.751595,0.721780,0.381388,0.501892,0.705630,0.545372,...,0.387600,0.641030,0.449715,0.576430,0.460896,0.772715,0.751595,0.592580,F3C8BBCD2DC26C1E0249DEEF6A4256B7,0
42369,0.114013,0.228026,0.180848,0.192643,0.302724,0.271272,0.174951,0.137602,0.241786,0.165122,...,0.127773,0.108116,0.096321,0.100253,0.155294,0.129739,0.076664,0.127773,A9A0FE83467A680FBFB0DBFC910DF227,0
42370,0.532178,0.469056,0.618879,0.543061,0.528913,0.553581,0.567366,0.789741,0.367119,0.421534,...,0.392513,0.296380,0.354785,0.323950,0.362766,0.396866,0.403033,0.296380,D9A6ADA018FA46A55D5438370456AA45,0


In [19]:
data_processed.to_csv("../data/02_intermediate/iot_sgcc_electricity_preprocessed.csv", index=False)

In [ ]:
# _test = pd.DataFrame({'val': [1,np.nan, 4, 5, np.nan, 10, np.nan, np.nan]})

# _test

In [ ]:
# _test["val"].fillna((_test.val.shift() + _test.val.shift(-1))/2)


# Train Test Split

### For GDN

In [20]:
from sklearn.model_selection import train_test_split

In [21]:
data_name = "sgcc_electricity_debug"

In [36]:
class_dist_dict = data_processed["FLAG"].value_counts(normalize=True).to_dict()

In [34]:
TRAIN_PC = 0.8
N_DEBUG = 2000 

n_train = int(TRAIN_PC * N_DEBUG)
n_test = int((1-TRAIN_PC) * N_DEBUG)
n_train, n_test

(1600, 399)

In [39]:
train_users = []
test_users = []

for _cls in class_dist_dict:
    n_train_cls = int(class_dist_dict[_cls] * n_train)
    n_test_cls = int(class_dist_dict[_cls] * n_test)
    
    users_cls = data_processed[data_processed["FLAG"] == _cls]["CONS_NO"].sample(n_train_cls+n_test_cls).values
    train_users.extend(users_cls[:n_train_cls])
    test_users.extend(users_cls[n_train_cls:])

In [40]:
len(train_users), len(test_users)

(1599, 398)

In [43]:
train_df = data_processed[data_processed["CONS_NO"].isin(train_users)]
test_df = data_processed[data_processed["CONS_NO"].isin(test_users)]

print(train_df.shape, test_df.shape)

train_df["FLAG"].value_counts(normalize=True), test_df["FLAG"].value_counts(normalize=True)

(1599, 1036) (398, 1036)


(0    0.914947
 1    0.085053
 Name: FLAG, dtype: float64,
 0    0.914573
 1    0.085427
 Name: FLAG, dtype: float64)

In [45]:
# train_df

In [46]:
pd.read_csv("../../GDN/data/msl/train.csv").head(100)

,timestamp,M-6,M-1,M-2,S-2,P-10,T-4,T-5,F-7,M-3,...,F-5,D-14,T-9,P-14,T-8,P-11,D-15,D-16,M-7,F-8
0,0,-1.0,0.999976,-0.748738,-1.0,0.994353,0.0,-1.0,-0.642857,1.000046,...,-0.059837,-1.0,-0.333329,0.999426,-1.0,0.941907,-1.000000,-1.000000,-1.001157,-0.826087
1,1,-1.0,0.999976,-0.748738,-1.0,0.993788,0.0,-1.0,-0.964286,1.000046,...,-0.059238,-1.0,-0.333329,0.999296,-1.0,0.944196,-1.000000,-1.000000,-1.001157,-0.869565
2,2,-1.0,0.999976,-0.748738,-1.0,0.994353,0.0,-1.0,-0.785714,1.000046,...,-0.059163,-1.0,-0.333329,0.999611,-1.0,0.943751,0.952800,0.983735,-1.001157,-0.869565
3,3,-1.0,0.999976,-0.748738,-1.0,0.993506,0.0,-1.0,-0.892857,1.000046,...,-0.058563,-1.0,-0.333329,0.999500,-1.0,0.941081,0.951874,-1.000000,-1.001157,-0.782609
4,4,-1.0,0.999976,-0.748738,-1.0,0.994353,0.0,-1.0,-0.964286,1.000046,...,-0.056016,-1.0,-1.000000,0.999519,-1.0,0.941653,-1.000000,-1.000000,-1.001157,-0.869565
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,-1.0,1.000000,-0.748717,-1.0,0.995482,-1.0,-1.0,-0.892857,1.000046,...,4.135299,-1.0,0.333338,0.999574,-1.0,0.958560,0.959278,0.999472,-1.001157,-0.782609
96,96,-1.0,1.000000,-0.748717,-1.0,0.992941,-1.0,-1.0,-0.892857,1.000046,...,4.135824,-1.0,0.333338,0.999648,-1.0,0.960276,0.958353,-1.000000,-1.001157,-1.000000
97,97,-1.0,0.786872,-1.101165,-1.0,0.993788,-1.0,-1.0,-0.892857,1.000046,...,4.136348,-1.0,0.333338,0.999574,-1.0,0.956399,0.957427,-1.000000,-1.001157,-0.956522
98,98,-1.0,0.787060,-1.101081,-1.0,0.994353,-1.0,-1.0,-0.892857,1.000046,...,4.138746,-1.0,0.333338,0.999444,-1.0,0.956272,0.955576,0.999472,-1.001157,-1.000000


In [47]:
pd.read_csv("../../GDN/data/msl/test.csv").head(100)

,timestamp,M-6,M-1,M-2,S-2,P-10,T-4,T-5,F-7,M-3,...,D-14,T-9,P-14,T-8,P-11,D-15,D-16,M-7,F-8,attack
0,0,-1.0,0.999906,-0.748738,-1.0,0.992941,0.0,-1.0,-0.892857,1.000000,...,-1.0,0.639967,0.992941,-1.0,0.930975,-1.000000,-1.0,-0.999133,-1.000000,0.0
1,1,-1.0,0.999906,-0.748738,-1.0,0.993788,0.0,-1.0,-0.892857,1.000000,...,-1.0,0.639967,0.993788,-1.0,0.918009,0.832485,-1.0,-0.999133,-0.565217,0.0
2,2,-1.0,0.999906,-0.748738,-1.0,0.993788,0.0,-1.0,-0.428571,1.000000,...,-1.0,0.639967,0.993788,-1.0,0.927861,0.831559,-1.0,-0.999133,-1.000000,0.0
3,3,-1.0,0.999906,-0.748738,-1.0,0.994635,0.0,-1.0,-0.500000,1.000000,...,-1.0,0.639967,0.994635,-1.0,0.921442,0.831559,-1.0,-0.999133,-1.000000,0.0
4,4,-1.0,0.999906,-0.748738,-1.0,0.994635,0.0,-1.0,-0.892857,1.000000,...,-1.0,0.637897,0.994635,-1.0,0.916484,0.831559,-1.0,-0.999133,-1.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,-1.0,0.999906,-0.748759,-1.0,0.993788,0.0,-1.0,-0.964286,1.000049,...,-1.0,0.639967,0.993788,-1.0,0.625004,-1.000000,-1.0,-0.999133,-1.000000,0.0
96,96,-1.0,0.999906,-0.748759,-1.0,0.993506,0.0,-1.0,-0.964286,1.000049,...,-1.0,0.637897,0.993506,-1.0,0.834938,-1.000000,-1.0,-0.999133,-1.000000,0.0
97,97,-1.0,0.999906,-0.748759,-1.0,0.993223,0.0,-1.0,-0.928571,1.000049,...,-1.0,0.639967,0.993223,-1.0,0.835637,0.918556,-1.0,-0.999133,-1.000000,0.0
98,98,-1.0,0.999906,-0.748759,-1.0,0.992376,0.0,-1.0,-0.928571,1.000049,...,-1.0,0.639967,0.992376,-1.0,0.836336,0.917631,-1.0,-0.999133,-1.000000,0.0


In [50]:
_test = pd.read_csv("../../GDN/data/msl/test.csv")
_test["attack"].value_counts()

1.0    1600
0.0     449
Name: attack, dtype: int64

In [52]:
_sensor_cols = [c for c in _test.columns if c not in ["timestamp", "attack"]]

In [54]:
_test[_sensor_cols].describe()

,M-6,M-1,M-2,S-2,P-10,T-4,T-5,F-7,M-3,M-4,...,F-5,D-14,T-9,P-14,T-8,P-11,D-15,D-16,M-7,F-8
count,2049.000000,2049.000000,2049.000000,2049.000000,2049.000000,2049.000000,2049.000000,2049.000000,2049.000000,2049.000000,...,2049.000000,2049.000000,2049.000000,2049.000000,2049.000000,2049.000000,2049.000000,2049.000000,2049.000000,2049.000000
mean,11.927255,0.551969,-0.152452,-0.888726,0.993711,-0.572474,-0.995398,-0.603500,0.437520,0.478622,...,-0.291479,-0.819982,0.346555,0.993711,-0.717820,0.788779,0.755457,-0.529394,-0.947370,-0.881830
std,37.594777,0.845124,0.857728,0.319171,0.000892,0.498772,0.093743,0.618148,0.726435,0.656647,...,0.644052,0.570227,0.373206,0.000892,0.474139,0.158032,0.654768,0.550896,0.313701,0.352459
min,-1.000000,-1.076476,-1.084689,-1.000000,0.990400,-1.000000,-1.000000,-1.000000,-1.424197,-1.331441,...,-1.076886,-1.000000,-1.000000,0.990400,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,-1.000000,0.429519,-0.830263,-1.000000,0.993223,-1.000000,-1.000000,-0.964286,0.188366,0.043704,...,-0.818727,-1.000000,0.000000,0.993223,-1.000000,0.695999,0.841740,-1.000000,-0.999602,-1.000000
50%,1.000000,0.579738,-0.748749,-1.000000,0.993788,-1.000000,-1.000000,-0.892857,0.697762,0.822859,...,-0.469969,-1.000000,0.579920,0.993788,-1.000000,0.829536,0.941694,-0.735361,-0.999602,-1.000000
75%,1.378378,0.999976,0.888108,-1.000000,0.994353,0.000000,-1.000000,-0.642857,1.000025,0.999995,...,-0.022669,-1.000000,0.639967,0.994353,0.000000,0.927607,1.054604,0.072534,-0.999602,-1.000000
max,258.108108,2.268854,1.335276,1.000000,0.998306,1.000000,1.000000,1.000000,1.000071,1.175893,...,1.806624,1.000000,1.000000,0.998306,1.019608,0.970509,1.346136,1.000000,1.000000,1.304348


In [48]:
pd.read_csv("../../GDN/data/msl/train.csv").shape

(1565, 28)

In [49]:
pd.read_csv("../../GDN/data/msl/test.csv").shape

(2049, 29)

In [22]:
a,b = train_test_split(data_processed["FLAG"])

In [23]:
a.shape, b.shape

((31779,), (10593,))

In [30]:
a.value_counts(normalize=True)

0    0.91422
1    0.08578
Name: FLAG, dtype: float64

In [32]:
b.value_counts(normalize=True)

0    0.916077
1    0.083923
Name: FLAG, dtype: float64